In [1]:
import numpy as np
import os
import glob
from astropy.io import fits
from astropy.table import Table, unique, Column, vstack
import matplotlib.pyplot as plt
from utils import *

#### Open DR12Q superset, DR12 spall and DR14 spall

In [2]:
f_sdr12q = '/global/projecta/projectdirs/sdss/data/sdss/dr12/boss/qso/DR12Q/Superset_DR12Q.fits'
sdr12q = fits.open(f_sdr12q)

In [3]:
f_spall_dr12 = '/global/projecta/projectdirs/sdss/data/sdss/dr12/boss/spectro/redux/spAll-DR12.fits'
spall_dr12 = fits.open(f_spall_dr12)

In [4]:
f_spall_dr14 = '/global/projecta/projectdirs/sdss/data/sdss/dr15/eboss/spectro/redux/v5_10_0/spAll-v5_10_0.fits'
spall_dr14 = fits.open(f_spall_dr14)

### Construct dictionaries mapping (p,m,f) to tid for both DRs

In [5]:
plate = spall_dr12[1].data["PLATE"]
mjd = spall_dr12[1].data["MJD"]
fid = spall_dr12[1].data["FIBERID"]
tid = spall_dr12[1].data["THING_ID"].astype(int)

pmf2tid_dr12 = {(p,m,f):t for p,m,f,t in zip(plate,mjd,fid,tid)}

In [ ]:
plate = spall_dr14[1].data["PLATE"]
mjd = spall_dr14[1].data["MJD"]
fid = spall_dr14[1].data["FIBERID"]
tid = spall_dr14[1].data["THING_ID"].astype(int)

pmf2tid_dr14 = {(p,m,f):t for p,m,f,t in zip(plate,mjd,fid,tid)}

#### Convert to a TID_DR12 -> TID_DR14 dictionary

In [7]:
boss2eboss = {pmf2tid_dr12[k]:pmf2tid_dr14[k] for k in pmf2tid_dr12.keys()}
eboss2boss = {pmf2tid_dr14[k]:pmf2tid_dr12[k] for k in pmf2tid_dr12.keys()}

In [8]:
tid_dr12 = np.array(list(boss2eboss.keys())).astype('int')
tid_dr14 = np.array(list(boss2eboss.values())).astype('int')

## Create a linked redux directory with targetid lists
I.e. replicate the BOSS redux directory structure with a directory per plate. Inside each directory is a link to the each of the DR12 spPlate files for that plate. This is only carried out for plates that have contain at least one QSO target.

Alongside these links are text files containing a list of targetids for that plate.

Also has the option to split data into several groups, in order to fit jobs into the debug queue (wrap-redrock takes a redux directory as an input rather than a list of spPlate files, so separate wrap-redrock runs need separate replicated redux directories).

In [37]:
# List of spplate files that we want to replicate. 
# NOTE: the same plate should not be listed more than once.
spplate_paths = glob.glob('/global/projecta/projectdirs/sdss/data/sdss/dr12/boss/spectro/redux/v5_7_?/*/spPlate*.fits')

# Set where the new redux replica will be stored.
new_redux_dir_base = '/global/cfs/projectdirs/desi/users/jfarr/BOSS_DR12_redux_replica_split/'
nsplits = 10


# Get a list of old redux dirs
old_redux_dirs = list(set([path.]))

# Make split dirs
split_dirs = [new_redux_dir_base+'/split_{}'.format(i) for i in range(nsplits)]

# Make replica redux dir if it doesn't exist already.
if not os.path.isdir(new_redux_dir_base):
    os.mkdir(new_redux_dir_base)
    
# Make split dirs if they don't exist already.
for split_dir in split_dirs:
    if not os.path.isdir(split_dir):
        os.mkdir(split_dir)

In [1]:
## Get the list of targetids for the objects in the superset.
wqso = np.in1d(spall_dr12[1].data["THING_ID"], sdr12q[1].data["THING_ID"]) & (spall_dr12[1].data["THING_ID"]>0)
plates = spall_dr12[1].data['PLATE'][wqso].astype('i8')
mjds = spall_dr12[1].data['MJD'][wqso].astype('i8')
fiberids = spall_dr12[1].data['FIBERID'][wqso].astype('i8')
targetids = platemjdfiber2targetid(plates,mjds,fiberids)
len(targetids)

NameError: name 'np' is not defined

In [39]:
# Get a list of unique plate-mjd pairs in the superset.
pm_set = set([(p,m) for p,m in zip(plates,mjds)])

# Check that each plate-mjd pair appears once and only once in the list of spPlates.
spplate_names = np.array([f.split('/')[-1] for f in spplate_paths])
for (p,m) in pm_set:
    name = 'spPlate-{}-{}.fits'.format(p,m)
    w = np.in1d(spplate_names,name)
    if w.sum()==0:
        print(name,'not found')
    elif w.sum()>1:
        print(name,'duplicated:')
        print(spplate_names[w])
        

In [40]:
# Divide the plates up into roughly even splits (making sure that spPlates from the same plate are in the same split).
plate_set = np.array(list(set(plates)))
sorted_plates = np.sort(plates)
plate_set.sort()
inds = np.array([np.argmax(np.in1d(plate_set,p)) for p in sorted_plates])
plate_splits = np.floor(nsplits*inds/(inds[-1]+1)).astype(int)
splits = {sorted_plates[i]:plate_splits[i] for i in range(len(sorted_plates))}

In [41]:
## FOR NOW, RESTRICT THE NUMBER OF PLATES
#plate_mod_100_vals = [35]
#pm_set = set([(pm[0],pm[1]) for pm in pm_set if (pm[0]//100 in plate_mod_100_vals)])

print(len(pm_set))

## For each plate, make a directory.
for p in plates:
    split = splits[p]
    plate_dir = split_dirs[split]+'/{}'.format(p)
    if not os.path.isdir(plate_dir):
        os.mkdir(plate_dir)

2493


In [44]:
## For each unique plate-mjd pair:
ntargets = {}
for i,(p,m) in enumerate(pm_set):
    
    ## Find which targetids correspond to that pair.
    w = (targetids//10000)==(platemjdfiber2targetid(p,m,0)//10000)
    pm_targetids = targetids[w]
    split = splits[p]
    dirname = split_dirs[split]+'/{}'.format(p)

    ## Write a txt file containing these targetids.
    with open(dirname+'/targetids-{}-{}.txt'.format(p,m),'w') as text_file:
        for t in pm_targetids[:-1]:
            text_file.write('{},'.format(t))
        text_file.write('{}'.format(pm_targetids[-1]))

    ntargets[(p,m)] = w.sum()
    
    print('Making targetid lists: {:.2%}'.format((i+1)/len(pm_set)),end='\r')
   
ntotal

        
    ## Make a symbolic link to the spPlate file from the old redux dir.
    name = 'spPlate-{}-{}.fits'.format(p,m)
    ind = np.argmax(np.in1d(spplate_names,name))
    src = spplate_paths[ind]
    dst = dirname+'/'.format(p)+name
    if not os.path.isfile(dst):
        os.symlink(src, dst)

    ## 
    src_dir = os.path.dirname(src)
    spcframes = glob.glob(src_dir+'/spCFrame*.fits')
    for spcframe in spcframes:

        ## Make symbolic links to all spCFrame files from the old redux dir (if they don't already exist).
        src = spcframe
        spcframe_file = spcframe.split('/')[-1]
        dst = dirname+'/'+spcframe_file
        if not os.path.isfile(dst):
            os.symlink(src, dst)

    
    print('Making targetid lists and spPlate/spCFrame replicas: {:.2%}'.format((i+1)/len(pm_set)),end='\r')
    
print('')
"""
## For each plate:
for i,p in enumerate(plates):
    
    split = splits[p]
    dirname = split_dirs[split]+'/{}'.format(p)
    
    ## For each redux dir:
    for old_redux_dir in old_redux_dirs:
        
        ## Find all spCFrame files.
        spcframes = glob.glob(old_redux_dir+'/{}/spCFrame*.fits'.format(p))
        for spcframe in spcframes:

            ## Make symbolic links to all spCFrame files from the old redux dir.
            src = spcframe
            spcframe_file = spcframe.split('/')[-1]
            dst = dirname+'/'.format(p)+spcframe_file
            os.symlink(src, dst)

    print('Making spCFrame links: {:.2%}'.format((i+1)/len(plates)),end='\r')

print('')"""

Making targetid lists and spPlate/spCFrame replicas: 100.00%


"\n## For each plate:\nfor i,p in enumerate(plates):\n    \n    split = splits[p]\n    dirname = split_dirs[split]+'/{}'.format(p)\n    \n    ## For each redux dir:\n    for old_redux_dir in old_redux_dirs:\n        \n        ## Find all spCFrame files.\n        spcframes = glob.glob(old_redux_dir+'/{}/spCFrame*.fits'.format(p))\n        for spcframe in spcframes:\n\n            ## Make symbolic links to all spCFrame files from the old redux dir.\n            src = spcframe\n            spcframe_file = spcframe.split('/')[-1]\n            dst = dirname+'/'.format(p)+spcframe_file\n            os.symlink(src, dst)\n\n    print('Making spCFrame links: {:.2%}'.format((i+1)/len(plates)),end='\r')\n\nprint('')"

### Merge the separate zbest files from redrock into a single file.

In [43]:
#rrdir = '/global/cfs/projectdirs/desi/users/jfarr/rrboss_dr12/runs/rr_v5_7_0/output/'
rrdir = '/global/cfs/projectdirs/desi/users/jfarr/rrboss_dr12/runs/rr_v5_7_0_ngpbestexp/output/'
f_out = '../results/rr_results/rr_sdr12q_ngpbestexp.fits'

if os.path.isfile(f_out):
    print('WARN: {} already exists, continuing will overwrite!'.format(f_out))
fi = glob.glob(rrdir+'/zbest*.fits')
print(len(fi))

WARN: ../results/rr_results/rr_sdr12q_ngpbestexp.fits already exists, continuing will overwrite!
2491


In [44]:
## Concatenate output from all zbest files.
rr = fits.open(fi[0])
rr_data = Table(rr[1].data)
rr.close()
if len(fi)>1:
    for i,f in enumerate(fi[1:]):
        rr = fits.open(f)
        f_rr_data = Table(rr[1].data)
        rr_data = vstack([rr_data,f_rr_data])
        rr.close()
        print('Stacking zbest files: {:.2%}'.format((i+2)/len(fi)),end='\r')
print('')
print(len(rr_data))
rr_data[:5]

Stacking zbest files: 100.00%
626180


TARGETID,CHI2,COEFF [10],Z,ZERR,ZWARN,NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,BRICKNAME
int64,float64,float64,float64,float64,int64,int64,str6,str20,int64,float64,str8
5021558630008,5637.105290252077,619.9783395824397 .. -19.25324332576535,-0.0024040552829318845,5.382775911979286e-05,4,4464,GALAXY,,10,3.0502007572777075,5021-558
5021558630009,6346.725192405283,0.0006964414537047506 .. 0.0,2.643902100086551,0.000623107033922408,0,4465,QSO,,4,2440.7694388777018,5021-558
5021558630010,7074.323110103607,95.77655068719827 .. -18.4433129809068,0.8094012310229814,2.6151080954427597e-05,0,4464,GALAXY,,10,32.89646768569946,5021-558
5021558630018,5655.046499311924,144.81654648756842 .. -24.626981379113683,1.0017471952358377,0.00010624680632263176,4,4456,GALAXY,,10,8.60011076927185,5021-558
5021558630022,6421.440160036087,15.582814296873796 .. -77.84759608253826,0.695642163830707,3.128383934292232e-05,0,4423,GALAXY,,10,53.41812264919281,5021-558


In [45]:
## Make sure that we only have objects that are in sdr12q.
plate,mjd,fiber = targetid2platemjdfiber(rr_data['TARGETID'])
pmf_rr = list(zip(plate,mjd,fiber))

In [46]:
pmf_sdr12q = list(zip(sdr12q[1].data['PLATE'],sdr12q[1].data['MJD'],sdr12q[1].data['FIBERID']))
wqso = np.in1d(spall_dr12[1].data["THING_ID"], sdr12q[1].data["THING_ID"]) & (spall_dr12[1].data["THING_ID"]>0)
plates = spall_dr12[1].data['PLATE'][wqso].astype('i8')
mjds = spall_dr12[1].data['MJD'][wqso].astype('i8')
fiberids = spall_dr12[1].data['FIBERID'][wqso].astype('i8')
targetid_sdr12q = platemjdfiber2targetid(plates,mjds,fiberids)

In [47]:
w = np.in1d(rr_data['TARGETID'],targetid_sdr12q)
print(w.sum(),len(w))
w = np.in1d(targetid_sdr12q,rr_data['TARGETID'])
print(w.sum(),len(w))

626180 626180
626180 627751


In [48]:
targetid_sdr12q = targetid_sdr12q[w]

In [49]:
rr_data.sort(['TARGETID'])
targetid_sdr12q.sort()

In [50]:
assert (rr_data['TARGETID'].data==targetid_sdr12q).all()

In [51]:
plate,mjd,fiber = targetid2platemjdfiber(rr_data['TARGETID'])
platemjdfiber = list(zip(plate,mjd,fiber))

In [52]:
rr_thing_id_dr12 = [pmf2tid_dr12[pmf] for pmf in platemjdfiber]

In [53]:
rr_data.add_column(Column(data=rr_thing_id_dr12,name='THING_ID_DR12',dtype='i8'))

In [54]:
rr_data.write(f_out,overwrite=True)

#### Open the redrock results (from v5_10_0), convert to DR12 TIDs and restrict to DR12Q superset objects

In [27]:
f_rr = '/project/projectdirs/desi/users/nbusca/QuasarNET/data/zbest_redrock_v5_10_0.fits'
#f_rr = '/project/projectdirs/desi/users/jfarr/rrboss_dr12/zbest/5786/zbestrr-5786-56251.fits'
rr = fits.open(f_rr)

In [28]:
def targetid2platemjdfiber(targetid):
    fiber = targetid % 10000
    mjd = (targetid // 10000) % 100000
    plate = (targetid // (10000 * 100000))
    return (plate, mjd, fiber)

In [29]:
def platemjdfiber2targetid(plate, mjd, fiber):
    return plate*1000000000 + mjd*10000 + fiber

In [30]:
plate,mjd,fiber = targetid2platemjdfiber(rr[1].data['TARGETID'])
pmf_rr = list(zip(plate,mjd,fiber))

In [31]:
pmf_sdr12q = list(zip(sdr12q[1].data['PLATE'],sdr12q[1].data['MJD'],sdr12q[1].data['FIBERID']))
targetid_sdr12q = platemjdfiber2targetid(sdr12q[1].data['PLATE'].astype('i8'),sdr12q[1].data['MJD'].astype('i8'),sdr12q[1].data['FIBERID'].astype('i8'))

In [43]:
w = np.in1d(rr[1].data['TARGETID'],targetid_sdr12q)
print(w.sum(),len(w))

546856 3907000


In [58]:
rr_data = rr[1].data
rr_data_table = Table(rr_data)[w]
rr_data_table[:5]

BRICKNAME,NCOEFF,CHI2,COEFF [4],TARGETID,ZWARN,SUBTYPE,ZERR,SPECTYPE,Z,NPIXELS,DELTACHI2
str8,int64,float64,float64,int64,int64,str20,float64,str6,float64,int64,float64
3586-551,4,4979.437905402494,0.2903345611768412 .. 0.08061822569416442,3586551810016,0,_3,0.0003939138858536805,QSO,2.2381091221699214,4182,385.92470588220294
3586-551,4,4559.26579044249,0.5035222657438391 .. 0.15087702032701958,3586551810032,0,G_13,6.675935901335709e-05,STAR,0.0006004932032965816,4182,12.294057197802431
3586-551,4,4672.553886816337,1.0266259688034056 .. 0.2529196190009618,3586551810036,0,A1III(225180)_10,3.654466257812394e-05,STAR,-0.00044412330005173405,4188,1385.840551828357
3586-551,4,13304.777461563503,0.3261850083074969 .. 1.477381964871323,3586551810038,0,_30,0.00025379391658500477,QSO,2.1699393767034914,4203,1081.5891517749878
3586-551,4,4680.414923773478,0.4113564338266882 .. -0.053412418891370335,3586551810040,0,B5(338529)_11,4.781706265572887e-05,STAR,-0.00045724101640536784,4167,89.69220200310974


In [59]:
rr_data_table.sort(['TARGETID'])
targetid_sdr12q.sort()

In [60]:
assert (rr_data_table['TARGETID']==targetid_sdr12q).all()

In [61]:
plate,mjd,fiber = targetid2platemjdfiber(rr_data_table['TARGETID'])
platemjdfiber = list(zip(plate,mjd,fiber))

In [62]:
rr_thing_id_dr12 = [pmf2tid_dr12[pmf] for pmf in platemjdfiber]

In [63]:
rr_data_table.add_column(Column(data=rr_thing_id_dr12,name='THING_ID_DR12',dtype='i8'))

In [66]:
rr_data_table.write('../results/rr_results/rr_sdr12q.fits',overwrite=True)

#### Open the qnAll file(s) we want, stitch them together and save.

In [31]:
fi_qn_loc = '/global/homes/j/jfarr/Projects/QuasarNET/test_mkl/' #'../results/qn_results//'
fi_qn_form = 'qnAll-qnet_0.1_1.fits'
fi_out = 'qn_qnet_0.1_1.fits'

In [32]:
import glob
fi_qn = glob.glob(fi_qn_loc+fi_qn_form)
data = []
for f_qn in fi_qn:
    qn = fits.open(f_qn)
    targetid_qn = platemjdfiber2targetid(qn[1].data['PLATE'].astype('i8'),qn[1].data['MJD'].astype('i8'),qn[1].data['FIBERID'].astype('i8'))
    w = (~(qn[1].data['IN_TRAIN'].astype('bool')))
    f_data = np.array(qn[1].data[w])
    data.append(f_data)
data = np.concatenate(data)
table = Table(data)
table[:5]

THING_ID,PLATE,MJD,FIBERID,ZBEST,IS_QSO,IN_TRAIN,C_LINES [6],Z_LINES [6],C_LINES_BAL,Z_LINES_BAL
int64,int32,int32,int32,float64,int64,int64,float64,float64,float64,float64
87897905,3586,55181,16,2.242729050049815,1,0,1.0 .. 2.9974560789014504e-07,2.242729050049815 .. -0.12054469464971318,2.0845945414293965e-07,2.2473561362279955
96906927,3586,55181,18,2.256086478630801,1,0,1.0 .. 9.466923529544147e-07,2.256086478630801 .. -0.05087198147299743,3.06321169318835e-07,4.8967792789364335
96907939,3586,55181,20,3.173956454313273,1,0,0.999936580657959 .. 3.83200540454709e-06,3.173513630730902 .. -0.07170452538796712,5.976783086225623e-07,5.054043491029256
87896919,3586,55181,32,2.3306701402016285,0,0,1.5827469042051234e-06 .. 7.1998162987085834e-09,2.3306701402016285 .. 0.4283017983938415,1.9319954525087724e-09,4.2836647859840955
87897024,3586,55181,36,0.5647317071789288,0,0,1.8513095483285724e-06 .. 2.8667849960584135e-07,5.383052623424779 .. 0.4610154015775567,1.0752962680271594e-06,1.8600473977283656


In [33]:
table_unique = unique(table, keys=['PLATE', 'MJD', 'FIBERID'])
table = fits.BinTableHDU(table_unique)
prihdr = fits.Header()
prihdu = fits.PrimaryHDU(header=prihdr)
hdulist = fits.HDUList([prihdu, table])
hdulist.writeto(fi_qn_loc+fi_out,overwrite=True)
hdulist.close()

### Make a directory structure with targetids in
For each (plate,mjd) pair, make a directory targetids/{plate}/, and create a file within it targetids-plate-mjd.txt

In [37]:
## Get all spectra for objects in DR12Q supserset
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)

In [39]:
## Get the set of (plate,mjd) pairs that cover these spectra
plate = spall_dr12[1].data['PLATE'][w].astype('i8')
mjd = spall_dr12[1].data['MJD'][w].astype('i8')
fiberid = spall_dr12[1].data['FIBERID'][w].astype('i8')

pm_set = set([(p,m) for p,m in zip(plate,mjd)])
print(len(pm_set))

2493


In [40]:
## Get the list of targetids for these spectra
targetids = platemjdfiber2targetid(plate,mjd,fiberid)

In [46]:
outpath = '/global/cfs/projectdirs/desi/users/jfarr/rrboss_dr12/'
for i,(p,m) in enumerate(pm_set):
    w = ((np.array(list(zip(plate,mjd)))==(p,m)).all(axis=1))
    pm_targetids = targetids[w]
    dirname = outpath+'/targetids/{}'.format(p)
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    with open(dirname+'/targetids-{}-{}.txt'.format(p,m),'w') as text_file:
        for t in pm_targetids[:-1]:
            text_file.write('{},'.format(t))
        text_file.write('{}'.format(pm_targetids[-1]))
    print('{:.2%}'.format((i+1)/len(pm_set)),end='\r')

### Remove duplicates from the SQUEzE dataset.

In [92]:
#f_sq = '/project/projectdirs/desi/users/jfarr/quasar_classifier_hack/Superset_DR12Q_SQUEzE.fits.gz'
f_sq = '/project/projectdirs/desi/users/iprafols/lya_working_group/QSO_classifiers/Superset_DR12Q_SQUEzE_v2.fits'
sq = fits.open(f_sq)
w = (~sq[1].data['duplicated'])
data = sq[1].data[w]
table = Table(data)
table[:5]

ra,dec,thing_id,plate,mjd,fiberid,z_vi,class_person,z_conf_person,boss_target1,ancillary_target1,ancillary_target2,eboss_target0,specid,z_true,loaded,ciii_ratio,ciii_ratio_SN,ciii_ratio2,civ_ratio,civ_ratio_SN,civ_ratio2,civ_blue_ratio,civ_blue_ratio_SN,civ_blue_ratio2,civ_red_ratio,civ_red_ratio_SN,civ_red_ratio2,ha_ratio,ha_ratio_SN,ha_ratio2,hb_ratio,hb_ratio_SN,hb_ratio2,lya_ratio,lya_ratio_SN,lya_ratio2,lyb_ratio,lyb_ratio_SN,lyb_ratio2,mgii_ratio,mgii_ratio_SN,mgii_ratio2,neiv_ratio,neiv_ratio_SN,neiv_ratio2,nev_ratio,nev_ratio_SN,nev_ratio2,oii_ratio,oii_ratio_SN,oii_ratio2,oiii_ratio,oiii_ratio_SN,oiii_ratio2,siiv_ratio,siiv_ratio_SN,siiv_ratio2,z_try,peak_significance,assumed_line,Delta_z,is_correct,is_line,correct_redshift,prob_class1,prob_class3,prob_class30,prob_class35,prob_class45,prob_class305,prob_class4,class_predicted,prob,duplicated
float64,float64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int64,int64,int64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str15,float64,bool,bool,bool,float64,float64,float64,float64,float64,float64,float64,int64,float64,bool
8.4677275669,0.38255848940000003,104282943,3586,55181,854,0.0,1,3,198656,0,0,0,104282943,0.0,False,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-1.0,-9999.99,none,-1.0,False,False,False,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,False
8.4693419224,-0.7643945646,77821070,3586,55181,180,0.0,1,3,65536,0,0,0,77821070,0.0,False,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-1.0,-9999.99,none,-1.0,False,False,False,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,False
8.4694132607,-0.8229059625,77821081,3586,55181,178,-1.0,0,1,1024,0,0,0,77821081,-1.0,False,0.9835209663000001,-0.2856364218,0.11586819270000001,1.2960948699000001,2.6796239288,0.029488872500000003,1.2664155724000001,1.8929070695,0.029488872500000003,1.3442229817,2.2653546122,0.029488872500000003,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,0.7642641137,-1.2736287082,0.0286050838,0.7928460155,-1.2260062513,0.2439940884,2.1703444494,1.3339470949,0.5831254085,1.1815662901,0.2727233012,0.4070429839,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,1.6075453118,7.1867165893,civ,2.6075453118,False,False,False,0.5167245392,0.0345601179,0.0192542632,0.1361913532,0.1797300308,0.1061420597,0.0073976361000000004,1,0.0538143811,False
8.4799282435,0.8723627653,116050950,3586,55181,880,2.442,3,3,1024,0,0,0,116050950,2.442,False,1.2360505403,5.2721498446,0.2829711679,1.7760673003,27.9774293462,0.06269671190000001,1.9205124795000001,26.2330073054,0.06269671190000001,1.5814675449,15.6790872179,0.06269671190000001,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,2.649487692,31.9623824282,0.0159946153,-9999.99,-9999.99,-9999.99,3.9856448989,4.8140782834,1.0555086704,0.8089575385000001,-1.4010136943,0.0663503029,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,-9999.99,1.1631070285,5.1585125741,0.1273431697,2.450517754,52.9805839329,lya,0.008517754,True

In [93]:
prihdr = fits.Header()
prihdu = fits.PrimaryHDU(header=prihdr)
table = fits.BinTableHDU(table)
hdulist = fits.HDUList([prihdu, table])
hdulist.writeto('../sq_results/sq_sdr12q.fits',overwrite=True)
hdulist.close()

### Read the pipeline classes/redshifts from DR12 spall, and put all results into a simplified file.

In [65]:
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)
w.sum()

627751

In [66]:
plate = spall_dr12[1].data['PLATE'][w].astype('i8')
mjd = spall_dr12[1].data['MJD'][w].astype('i8')
fiberid = spall_dr12[1].data['FIBERID'][w].astype('i8')

targetid = platemjdfiber2targetid(plate,mjd,fiberid)
spectype = spall_dr12[1].data['CLASS'][w]
z = spall_dr12[1].data['Z'][w]
zwarn = spall_dr12[1].data['ZWARNING'][w]

thing_id = [pmf2tid_dr12[(p,m,f)] for p,m,f in zip(plate,mjd,fiberid)]

data_list = list(zip(targetid,spectype,z,zwarn,thing_id))
dtype = [('TARGETID','i8'),('SPECTYPE','U8'),('Z','f8'),('ZWARN','i8'),('THING_ID_DR12','i8')]

data = np.array(data_list,dtype=dtype)

In [67]:
table = Table(data)
table[:5]

TARGETID,SPECTYPE,Z,ZWARN,THING_ID_DR12
int64,str8,float64,int64,int64
3586551810016,QSO,2.2423312664031982,0,87897905
3586551810018,QSO,2.259599208831787,0,96906927
3586551810020,QSO,3.1812267303466797,0,96907939
3586551810032,STAR,-0.0031940187327563763,4,87896919
3586551810036,STAR,-0.00046957170707173645,0,87897024


In [68]:
table.write('../dr12pipe_results/dr12pipe_sdr12q.fits',overwrite=True)

### Do the same for DR14

In [82]:
w = np.in1d(spall_dr12[1].data['THING_ID'],sdr12q[1].data['THING_ID'])
w &= (spall_dr12[1].data['THING_ID']!=-1)
w.sum()

627751

In [83]:
targetid_sdr12q = platemjdfiber2targetid(spall_dr12[1].data['PLATE'][w],spall_dr12[1].data['MJD'][w],spall_dr12[1].data['FIBERID'][w])
targetid_dr14 = platemjdfiber2targetid(spall_dr14[1].data['PLATE'],spall_dr14[1].data['MJD'],spall_dr14[1].data['FIBERID'])
w = np.in1d(targetid_dr14,targetid_sdr12q)

In [85]:
plate = spall_dr14[1].data['PLATE'][w].astype('i8')
mjd = spall_dr14[1].data['MJD'][w].astype('i8')
fiberid = spall_dr14[1].data['FIBERID'][w].astype('i8')

targetid = platemjdfiber2targetid(plate,mjd,fiberid)
spectype = spall_dr14[1].data['CLASS'][w]
z = spall_dr14[1].data['Z'][w]
zwarn = spall_dr14[1].data['ZWARNING'][w]

thing_id = [pmf2tid_dr12[(p,m,f)] for p,m,f in zip(plate,mjd,fiberid)]

data_list = list(zip(targetid,spectype,z,zwarn,thing_id))
dtype = [('TARGETID','i8'),('SPECTYPE','U8'),('Z','f8'),('ZWARN','i8'),('THING_ID_DR12','i8')]

data = np.array(data_list,dtype=dtype)

In [86]:
table.write('../dr14pipe_results/dr14pipe_sdr12q.fits',overwrite=True)